## NYC Parking Ticket - EDA - Case Study


## 1. Objectives
>- The objective of this case study is to do exploratory data analysis using pyspark and drawing inference from the data, in big data clustered distributed environment. Also one of the main objective is to gain familiarity with how data analysis works in pyspark.<br><br>
>- Our Analysis is focussed on performing exploratory data analysis to understand the parking violation dataset. We will compare metrics/phenomenon related to Parking Tickets for the fiscal year 2017. The Analysis will be performed using Pyspark running in corestack with data in Hadoop HDFS Cluster.<br>

## 2.  Data Description
>- The data for this case study has been placed in HDFS at the following path: `'/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'`<br>

> The Data Dictionary for this dataset is as follows: (Field Name (Type) --Description  )<br>
1.  `Summons Number     - (long)`     : Unique Identifier Of Summons (Primary Key)  
2.  `Plate ID           - (string)`   : Registered Plate Id
3.  `Registration State - (string)`   : State Of Plate Registration
4.  `Issue Date         - (timestamp)`: Issue Date 
5.  `Violation Code     - (integer)`  : Type Of Violation
6.  `Vehicle Body Type  - (string)`   : Vehicle Body Type Written On Summons
7.  `Vehicle Make       - (string)`   : Make Of Car Written On Summons
8.  `Violation Precinct - (integer)`  : Percinct Of Violation
9.  `Issuer Precinct    - (integer)`  : Precinct Of Issuance
10. `Violation Time     - (string)`   : Time Violation Occurred

### Assumptions<br>
>1. The dataset under folder `/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv` for this case study has tickets issued from year 1972. It is assumed that for this case study, we are considering only 2017 year data, the rest of the year data has been filtered and been removed.

>2. The attribute names in the dataset contain white spaces and special characters. To facilitate ease of analysis we have replaced the white spaces with “_” and removed special characters.

>3. The date and time formats are not in a standard format therefore we have conducted data manipulation to extract the required information as per the requirement.
 
>4. We have used the details from NYC Department of finance for Violation of code, fines details for The Ticket Fine revenue estimation with the help of the following Source:  http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page

>5. The Seasonality vs. Ticket Frequency analysis was performed by creating buckets for the season with reference to the ticket’s Issue_Date [i.e, Issue_Month]. 
    - The seasons were defined with an assumption as as follows:<br> 
         a. Spring runs from March 1 to May 31; <br>
         b. Summer runs from June 1 to August 31; <br>
         c. Fall (autumn) runs from September 1 to November 30; and,<br>
         d. Winter runs from December 1 to February 28.<br>

## 3. Exploratory Data Analysis.

### Stage 1:  Data Quality Verification and Cleaning

#### 1.1 Initialise the Spark Session.

In [1]:
# Set the Application Spark Session.
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("NYC Parking Ticket Application") \
    .getOrCreate()

#### 1.2 Import the NYC Parking 2017 dataset into a dataframe.

In [2]:
nyc_parking_tkt_2017 = spark.read.format("csv") \
                        .option("header","true") \
                        .option("inferschema","true") \
                        .load('/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv') \
                        .coalesce(5)

In [3]:
# cache dataframe for better performance
nyc_parking_tkt_2017.cache()

DataFrame[Summons Number: bigint, Plate ID: string, Registration State: string, Issue Date: timestamp, Violation Code: int, Vehicle Body Type: string, Vehicle Make: string, Violation Precinct: int, Issuer Precinct: int, Violation Time: string]

#### 1.3 Verify Dimensions, Schema and Data.

In [4]:
# print number of rows & columns in the dataframe.
print("Shape: ", (nyc_parking_tkt_2017.count(),len(nyc_parking_tkt_2017.columns)))

Shape:  (10803028, 10)


In [5]:
# print the schema of the dataframe.
nyc_parking_tkt_2017.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)



In [6]:
# print top 5 rows.
nyc_parking_tkt_2017.show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

***Observations***<br>
 - The comma delimeted csv file `NYC Parking violation data for year 2017` is imported in to a dataframe.
 - There are `10,803,028` rows and `10` columns in the dataframe.
 - It is observed that there are issues in the column names.
     - There are spaces before and after the column names.
     - The column names have spaces in between the words which is NOT conformable for SQL queries.<br>
     
***Inference***<br>
 - The column names should be fixed before creating view Table. 

#### 1.4 Fix Column Names of DataFrame.

In [7]:
from pyspark.sql.functions import *
#from pyspark.sql.functions import col

# Trim the column Names and replaces spaces between words in the column Names with underscore (_)
for col_name in nyc_parking_tkt_2017.columns:
    nyc_parking_tkt_2017 = nyc_parking_tkt_2017.withColumnRenamed(col_name,col_name.replace(" ", "_").strip())

# Check the column Names.
nyc_parking_tkt_2017.columns

['Summons_Number',
 'Plate_ID',
 'Registration_State',
 'Issue_Date',
 'Violation_Code',
 'Vehicle_Body_Type',
 'Vehicle_Make',
 'Violation_Precinct',
 'Issuer_Precinct',
 'Violation_Time']

> ***Observations***<br>
 - The column names in the dataframe has white spaces and also leading and trailing whitespaces
 - The column Names are trimmed and words in the column name delimited with underscore (_).

In [8]:
nyc_parking_tkt_2017.select('Summons_Number','Issue_Date','Violation_Time').show(10)


+--------------+-------------------+--------------+
|Summons_Number|         Issue_Date|Violation_Time|
+--------------+-------------------+--------------+
|    5092469481|2016-07-10 00:00:00|         0143A|
|    5092451658|2016-07-08 00:00:00|         0400P|
|    4006265037|2016-08-23 00:00:00|         0233P|
|    8478629828|2017-06-14 00:00:00|         1120A|
|    7868300310|2016-11-21 00:00:00|         0555P|
|    5096917368|2017-06-13 00:00:00|         0852P|
|    1413609545|2016-08-03 00:00:00|         0215A|
|    4628525523|2016-12-21 00:00:00|         0758A|
|    4627113330|2016-11-21 00:00:00|         1005A|
|    4006478550|2016-10-05 00:00:00|         0845A|
+--------------+-------------------+--------------+
only showing top 10 rows



#### 1.5 Duplicate Rows based on same Summons Numbers.

In [9]:
# Drop any duplicate rows which is having the same Summons_Number.
nyc_parking_tkt_2017 = nyc_parking_tkt_2017.dropDuplicates(['Summons_Number'])

# Verify Nodup dataframe.
print("Shape: ",(nyc_parking_tkt_2017.count(),len(nyc_parking_tkt_2017.columns)))

Shape:  (10803028, 10)


> ***Observations***<br>
 - There are no duplicates in the data frame having duplicate summons Number. 

#### 1.6 Create TemporaryTableView for the Dataframe for SQL Queries.

In [10]:
nyc_parking_tkt_2017.createOrReplaceTempView("tbl_nyctkt_2017")

> ***Observations***<br>
> - A new Temporary View Table `tbl_nyctkt_2017` is created for analysis using SQL Queries.

#### 1.7 Verify Nulls in the columns

In [11]:
# Verify for NULL values in the data frame using functions.
nyc_parking_tkt_2017.select([count(when(col(c).isNull(), c)).alias(c) \
                                   for c in nyc_parking_tkt_2017.columns]).show()

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons_Number|Plate_ID|Registration_State|Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|             0|       0|                 0|         0|             0|                0|           0|                 0|              0|             0|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+



In [12]:
sql_1 = "SELECT \
             SUM(CASE WHEN Plate_ID IS NULL THEN 1 ELSE 0 END)               AS pid_nulls,\
             SUM(CASE WHEN Registration_State IS NULL THEN 1 ELSE 0 END)     AS rstate_nulls,\
             SUM(CASE WHEN Issue_Date IS NULL THEN 1 ELSE 0 END)             AS idate_nulls,\
             SUM(CASE WHEN Vehicle_Body_Type IS NULL THEN 1 ELSE 0 END)      AS vbtype_nulls,\
             SUM(CASE WHEN Vehicle_Make IS NULL THEN 1 ELSE 0 END)           AS vm_nulls,\
             SUM(CASE WHEN Violation_Time IS NULL THEN 1 ELSE 0 END)         AS vtime_nulls\
         FROM \
             tbl_nyctkt_2017 \
         WHERE \
            (Plate_ID IS NULL            OR \
             Registration_State IS NULL  OR \
             Issue_Date IS NULL          OR \
             Vehicle_Body_Type IS NULL   OR \
             Vehicle_Make IS NULL        OR \
             Violation_Time IS NULL) " 

spark.sql(sql_1).show()

+---------+------------+-----------+------------+--------+-----------+
|pid_nulls|rstate_nulls|idate_nulls|vbtype_nulls|vm_nulls|vtime_nulls|
+---------+------------+-----------+------------+--------+-----------+
|     null|        null|       null|        null|    null|       null|
+---------+------------+-----------+------------+--------+-----------+



> ***Observations***<br>
 - There are no records with nulls for `Plate_ID,Registration_State, Issue_Date, Vechicle_Body_Type,Vehicle_Make and Violation_Time`.

#### 1.8 Verify Issue_Date Column for the Range Values.
>- Since ticket Issue Date is an critical attribute to assess the quality of the Dataset.
Let us check if there are any missing values in the Issue Date column attribute.

In [13]:
# Find the minimum & maximum issue_date range. to check for the 2017 records.
sql_2="SELECT MIN(Issue_Date) AS Min_IssueDate_2017, MAX(Issue_Date) AS Max_IssueDate_2017\
       FROM tbl_nyctkt_2017" 

spark.sql(sql_2).show()

+-------------------+-------------------+
| Min_IssueDate_2017| Max_IssueDate_2017|
+-------------------+-------------------+
|1972-03-30 00:00:00|2069-11-19 00:00:00|
+-------------------+-------------------+



> ***Observations***<br>
 - The Issue Tickets range between 30th March 1972 to 19th November 2069. Clearly this is Nonconforming. Let us Analyse column closely to decide optimal filter condition.
 - We will create two additional column issue_year and issue_month to create a new data frame.
 - Observe the distribution of year & month based on Issue_Date.

#### 1.9 Create Issue_Year & Issue_Month from Issue_Date Column

In [14]:
# Create Additional Columns in the data frame Issue_Year and Issue_Month
nyc_parking_tkt_2017 = nyc_parking_tkt_2017.withColumn("Issue_Year",year(nyc_parking_tkt_2017.Issue_Date))
nyc_parking_tkt_2017 = nyc_parking_tkt_2017.withColumn("Issue_Month",month(nyc_parking_tkt_2017.Issue_Date))

#print the new columns.
nyc_parking_tkt_2017.select("Summons_Number","Issue_Date","Issue_Year","Issue_Month").show(5)

+--------------+-------------------+----------+-----------+
|Summons_Number|         Issue_Date|Issue_Year|Issue_Month|
+--------------+-------------------+----------+-----------+
|    8539607141|2017-05-05 00:00:00|      2017|          5|
|    8491667738|2017-03-01 00:00:00|      2017|          3|
|    5094058873|2016-10-29 00:00:00|      2016|         10|
|    8514570973|2017-06-21 00:00:00|      2017|          6|
|    1414975570|2016-10-04 00:00:00|      2016|         10|
+--------------+-------------------+----------+-----------+
only showing top 5 rows



#### 1.10 Re-Create Table View

In [15]:
# Re Create a Temporary View for analysis.
nyc_parking_tkt_2017.createOrReplaceTempView("tbl_nyctkt_2017")

#### 1.11 Distribution of Year & Month of Issue_Date

In [16]:
##### Observe the Distribution of Issue Date.
sql_3="SELECT Issue_Year, Issue_Month, COUNT(*) AS num_records \
       FROM tbl_nyctkt_2017 \
       WHERE ISSUE_Year =2017 \
       GROUP BY Issue_Year, Issue_Month \
       ORDER BY 1,2" 

spark.sql(sql_3).show(12)

+----------+-----------+-----------+
|Issue_Year|Issue_Month|num_records|
+----------+-----------+-----------+
|      2017|          1|     877365|
|      2017|          2|     826967|
|      2017|          3|     964737|
|      2017|          4|     888402|
|      2017|          5|    1020244|
|      2017|          6|     852187|
|      2017|          7|        370|
|      2017|          8|        309|
|      2017|          9|        367|
|      2017|         10|        274|
|      2017|         11|        338|
|      2017|         12|        358|
+----------+-----------+-----------+



> ***Observations***<br>
- It is clear from the above query the parking Tickets have been Issued in year 2017 are between January, 2017 and December,2017.
> - Filter the rows only required year 2017 for further Analysis.
> - So, the total number of records after filtering for year 2017 is `5,431,918`.

> ***Inference***<br>
> - We will filtered data frame - the 2017  year - for our further ED Analysis.

#### 1.12 Filter rows based on Issue Date (Year & Month) for year 2017.

In [17]:
# Filter 2017 Fiscal Year Data.
nyc_parking_tkt_2017_final = nyc_parking_tkt_2017\
                                  .filter((col("Issue_Year") == 2017) & \
                                          ((col("Issue_Month") >= 1)  | (col("Issue_Month") <= 12))
                                          ) 

# Check Dimensions.
print("Shape: ",(nyc_parking_tkt_2017_final.count(),len(nyc_parking_tkt_2017_final.columns)))

Shape:  (5431918, 12)


In [18]:
# Re Create a Temporary View for analysis.
nyc_parking_tkt_2017_final.createOrReplaceTempView("tbl_nyctkt_2017")

### Stage 2 : Overview and Examine the data

#### 2.1 Question-1: Find the total number of tickets for the year 

#### 2.1.1 Query for total Number of Tickets Received across all states in Year 2017.

In [19]:
# Get the total number of ickets issued during year 2017.

# SQL Query - sql_4
sql_4 = "SELECT count(*) as Num_Tickets_2017 FROM tbl_nyctkt_2017"

#Execute the Query.
spark.sql(sql_4).show()

+----------------+
|Num_Tickets_2017|
+----------------+
|         5431918|
+----------------+



#### Q1: Answer :
> - The total number of Tickets issued for the fiscal year 2017 is `5,431,918`.

#### 2.2 Question-2: Find out the number of unique states from where the cars that got parking tickets came. (Hint: Use the column 'Registration State'.)

#### 2.2.1 Query for number of unique States which received maximum tickets.

In [20]:
# To find the maximum entries per registration state in Year 2017

# SQL Query - sql_5
sql_5="SELECT Registration_State, COUNT(*) AS Number_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE year(Issue_Date) = 2017 \
       GROUP BY Registration_State \
       ORDER BY Number_of_Tickets DESC"

#Execute the Query.
sql_5_result = spark.sql(sql_5) 
sql_5_result.show(5)

+------------------+-----------------+
|Registration_State|Number_of_Tickets|
+------------------+-----------------+
|                NY|          4273951|
|                NJ|           475825|
|                PA|           140286|
|                CT|            70403|
|                FL|            69468|
+------------------+-----------------+
only showing top 5 rows



In [21]:
# Check the number of unique entries.
print("Number of Unique Registration States:",sql_5_result.count())

Number of Unique Registration States: 65


***Observations***<br>

- From the initial analysis the number of unique states are `65`
- There is an incorrect numerical entry `99`, which should be replaced with the state having maximum number of tickets.
- It is clear from the above query results maximum number of registration entries are from `NY` State.

#### 2.2.2 Query for the number of incorrect entry '99'

In [22]:
# Find number of entries with incorrect entry '99' in Registration State

# SQL Query - sql_5
sql_6="SELECT \
        COUNT(*) AS Number_of_99_States \
       FROM tbl_nyctkt_2017 \
       WHERE (Registration_State = 99 AND year(Issue_Date) =2017)"

#Execute the Query.
spark.sql(sql_6).show()

+-------------------+
|Number_of_99_States|
+-------------------+
|              16055|
+-------------------+



***Observations***<br>
>- There are `16055` entries with Numeric data `99` in  the Registration_State which needs to be replaced with `NY`.

#### 2.2.3 Replace the entry '99' with 'NY' in the data frame 

In [23]:
# Replacing in Original Dataframe for Registration_State where Numeric data 99 with 'NY'.
from pyspark.sql import functions as F
nyc_parking_tkt_2017_final = nyc_parking_tkt_2017_final \
                                    .withColumn("Registration_State", F.when(F.col("Registration_State") == 99, 'NY') \
                                    .otherwise(F.col("Registration_State")))

In [24]:
# Check all 99 columns is replaced.
nyc_parking_tkt_2017_final.where(col("Registration_State") == 99).count()

0

#### 2.2.4 Create Temporary View Table.

In [25]:
# Updating the View after replacing 99 in Registration State with 'NY'
nyc_parking_tkt_2017_final.createOrReplaceTempView("tbl_nyctkt_2017")

#### 2.2.5 Rerun Query for number of unique States which received maximum tickets.

In [26]:
# Find the maximum entries per registration state in YearR 2017

# SQL Query - sql_8
sql_8="SELECT Registration_State, COUNT(*) AS Number_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE year(Issue_Date) =2017 \
       GROUP BY Registration_State \
       ORDER BY Number_of_Tickets DESC"

#Execute the Query.
sql_8_result = spark.sql(sql_8)
sql_8_result.show(5)

+------------------+-----------------+
|Registration_State|Number_of_Tickets|
+------------------+-----------------+
|                NY|          4290006|
|                NJ|           475825|
|                PA|           140286|
|                CT|            70403|
|                FL|            69468|
+------------------+-----------------+
only showing top 5 rows



#### 2.2.6 Verify Results.

In [27]:
# Check the number of unique entries post cleanup 99 entries.

print("Number of Unique Registration States:",sql_8_result.count())

Number of Unique Registration States: 64


#### Q2: Answer : :
> - All incorrect `99` entries in Registration_State column have been replaced with maximum entry state value `NY`.
> - Rerun the query again to find the entries for each registration states.
> - The number of unique states from where the cars got parking tickets are `64`.
> - The maximum number of tickets are received in the state of `NY`.

### Stage 3 : Aggregation tasks

#### 3.1 Question-1: How often does each violation code occur? Display the frequency of the top five violation codes.

#### 3.1.1 Query for Top 5 Violation Codes (based on Frequency of Tickets)

In [28]:
# Find the Top 5 Violation Cods for 2017.
# SQL Query - sql_9
sql_9="SELECT DISTINCT(Violation_Code), COUNT(*) AS Number_of_Violations \
       FROM tbl_nyctkt_2017 \
       GROUP BY Violation_Code \
       ORDER BY Number_of_Violations DESC"

#Execute the Query and list out top 5.
sql_9_result = spark.sql(sql_9)
sql_9_result.show(5)

+--------------+--------------------+
|Violation_Code|Number_of_Violations|
+--------------+--------------------+
|            21|              768087|
|            36|              662765|
|            38|              542079|
|            14|              476664|
|            20|              319646|
+--------------+--------------------+
only showing top 5 rows



#### Stage3: Q1: Answer : :
> - The **`Top 5 Violation Codes and the frequencies`** are as below ;<br>
        +--------------+--------------------+ 
        Violation_Code |Number_of_Violations| 
        +--------------+--------------------+ 
        |            21|              768087| 
        |            36|              662765| 
        |            38|              542079| 
        |            14|              476664| 
        |            20|              319646| 
        +--------------+--------------------+ 

#### 3.2: Question-2: How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? (Hint: Find the top 5 for both.)

#### 3.2.1 Query for Top 5 Vehicle body Type for the Year 2017 (based on Frequency of Tickets)

In [29]:
# Find the Top 5 Vehicle Body Type for 2017

# SQL Query - sql_10
sql_10="SELECT DISTINCT(Vehicle_Body_Type), COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       GROUP BY Vehicle_Body_Type \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out top 5.
sql_10_result = spark.sql(sql_10)
sql_10_result.show(5)

+-----------------+--------------------+
|Vehicle_Body_Type|Frequency_of_Tickets|
+-----------------+--------------------+
|             SUBN|             1883954|
|             4DSD|             1547312|
|              VAN|              724029|
|             DELV|              358984|
|              SDN|              194197|
+-----------------+--------------------+
only showing top 5 rows



#### 3.2.2 Query for Top 5 Vehicle Make for the Year 2017 (based on Frequency of Tickets)

In [30]:
# Find the Top 5 Vehicle Make for 2017

# SQL Query - sql_11
sql_11="SELECT DISTINCT(Vehicle_Make), COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       GROUP BY Vehicle_Make \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out top 5.
sql_11_result = spark.sql(sql_11)
sql_11_result.show(5)

+------------+--------------------+
|Vehicle_Make|Frequency_of_Tickets|
+------------+--------------------+
|        FORD|              636844|
|       TOYOT|              605291|
|       HONDA|              538884|
|       NISSA|              462017|
|       CHEVR|              356032|
+------------+--------------------+
only showing top 5 rows



#### Stage3: Q2: Answer : :
> - The **`Top 5 Vehicle Body Type for 2017 and the frequencies of tickets`** are as below ;<br>
        +-----------------+--------------------+
        |Vehicle_Body_Type|Frequency_of_Tickets|
        +-----------------+--------------------+
        |             SUBN|             1883954|
        |             4DSD|             1547312|
        |              VAN|              724029|
        |             DELV|              358984|
        |              SDN|              194197|
        +-----------------+--------------------+
>- The **`Top 5 Vehicle Make for 2017 and the frequencies of tickets`** are as below ;<br>
        +------------+--------------------+
        |Vehicle_Make|Frequency_of_Tickets|
        +------------+--------------------+
        |        FORD|              636844|
        |       TOYOT|              605291|
        |       HONDA|              538884|
        |       NISSA|              462017|
        |       CHEVR|              356032|
        +------------+--------------------+

#### 3.3: Question-3:  A precinct is a police station that has a certain zone of the city under its command. Find the (5 highest) frequencies of tickets for each of the following;
>- #### 1. 'Violation Precinct' (This is the precinct of the zone where the violation occurred). Using this, can you draw any insights for parking violations in any specific areas of the city?  
> - #### 2. 'Issuer Precinct' (This is the precinct that issued the ticket.)
> - #### Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'. These are erroneous entries. Hence, you need to provide the records for five correct precincts. (Hint: Print the top six entries after sorting.)

#### 3.3.1 Query for Top 5 Violation Precinct (based on Frequency of Tickets)

In [31]:
# 1. Violation Precinct vs Frequency of Tickets.- 

# Find the Top 5 Violation Precinct for 2017

# SQL Query - sql_12
sql_12="SELECT DISTINCT(Violation_Precinct), COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE Violation_Precinct <> 0 \
       GROUP BY Violation_Precinct \
       ORDER BY Frequency_of_Tickets DESC"


#Execute the Query and list out top 5.
sql_12_result = spark.sql(sql_12)
sql_12_result.show(5)

+------------------+--------------------+
|Violation_Precinct|Frequency_of_Tickets|
+------------------+--------------------+
|                19|              274445|
|                14|              203553|
|                 1|              174702|
|                18|              169131|
|               114|              147444|
+------------------+--------------------+
only showing top 5 rows



#### 3.3.2 Query for Top 5 Issuer Precinct (based on Frequency of Tickets)

In [32]:
# Issuer Precinct vs Frequency of Tickets:-

# Find the Top 5 Issuer Precinct for 2017.
# SQL Query - sql_13
sql_13="SELECT DISTINCT(Issuer_Precinct), COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE Issuer_Precinct <> 0 \
       GROUP BY Issuer_Precinct \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out top 5.
sql_13_result = spark.sql(sql_13)
sql_13_result.show(5)

+---------------+--------------------+
|Issuer_Precinct|Frequency_of_Tickets|
+---------------+--------------------+
|             19|              266961|
|             14|              200495|
|              1|              168740|
|             18|              162994|
|            114|              144054|
+---------------+--------------------+
only showing top 5 rows



#### Stage3: Q3: Answer : :
> - 1. The **`Top 5 Violation Precinct for 2017 and the frequencies of tickets`** are as below ;<br>
            +------------------+--------------------+
            |Violation_Precinct|Frequency_of_Tickets|
            +------------------+--------------------+
            |                19|              274445|
            |                14|              203553|
            |                 1|              174702|
            |                18|              169131|
            |               114|              147444|
            +------------------+--------------------+
> - 2. The **`Top 5 Issuer Precinct for 2017 and the frequencies of tickets`** are as below ;<br>
            +---------------+--------------------+
            |Issuer_Precinct|Frequency_of_Tickets|
            +---------------+--------------------+
            |             19|              266961|
            |             14|              200495|
            |              1|              168740|
            |             18|              162994|
            |            114|              144054|
            +---------------+--------------------+

#### 3.4: Question-4: Find the violation code frequencies for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts? 
>- #### (Hint: In the SQL view, use the 'where' attribute to filter among three precincts.)

#### 3.4.1 Query the Voilation Code Frequencies for three Top Precints (19,14 and 1)

In [33]:
# Violation Code Distribution in Top three Issuer precincts.

# Find the Top 3 Violation Codes for Issuer precients in the year 2017.

# Note: 
#  The Top Three issuer precincts based on the frequency of the tickets (as per the question 3) are 19, 14 and 1.
#

# Violation Codes for the Issuer precinct 19.
# SQL Query - sql_14
sql_14="SELECT DISTINCT(Violation_Code), Issuer_Precinct, COUNT(*)  AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE Issuer_Precinct = 19  \
       GROUP BY Issuer_Precinct, Violation_Code \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out top 5.
sql_14_result = spark.sql(sql_14)
sql_14_result.show(5)


# Violation Codes for the Issuer precinct 14.
# SQL Query - sql_15
sql_15="SELECT DISTINCT(Violation_Code), Issuer_Precinct, COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE Issuer_Precinct = 14  \
       GROUP BY Issuer_Precinct, Violation_Code  \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out top 5.
sql_15_result = spark.sql(sql_15)
sql_15_result.show(5)


# Violation Codes for the Issuer precinct 1.
# SQL Query - sql_16
sql_16="SELECT DISTINCT(Violation_Code), Issuer_Precinct, COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE Issuer_Precinct = 1  \
       GROUP BY Issuer_Precinct, Violation_Code \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out top 5.
sql_16_result = spark.sql(sql_16)
sql_16_result.show(5)

+--------------+---------------+--------------------+
|Violation_Code|Issuer_Precinct|Frequency_of_Tickets|
+--------------+---------------+--------------------+
|            46|             19|               48445|
|            38|             19|               36386|
|            37|             19|               36056|
|            14|             19|               29797|
|            21|             19|               28415|
+--------------+---------------+--------------------+
only showing top 5 rows

+--------------+---------------+--------------------+
|Violation_Code|Issuer_Precinct|Frequency_of_Tickets|
+--------------+---------------+--------------------+
|            14|             14|               45036|
|            69|             14|               30464|
|            31|             14|               22555|
|            47|             14|               18364|
|            42|             14|               10027|
+--------------+---------------+--------------------+
onl

#### 3.4.2 Query the Other Violation and their Frequency for Comparison (other than 19,14 and 1)

In [34]:
# Violation Codes for the Issuer precinct other than 19,14 and 1.
# SQL Query - sql_17
sql_17="SELECT DISTINCT(Violation_Code), COUNT(*) AS Frequency_of_Tickets \
       FROM tbl_nyctkt_2017 \
       WHERE Issuer_Precinct NOT IN (19,14,1)  \
       GROUP BY Violation_Code \
       ORDER BY Frequency_of_Tickets DESC"

#Execute the Query and list out .
sql_17_result = spark.sql(sql_17)
sql_17_result.show(5)

+--------------+--------------------+
|Violation_Code|Frequency_of_Tickets|
+--------------+--------------------+
|            21|              734588|
|            36|              662765|
|            38|              493889|
|            14|              363477|
|            20|              286848|
+--------------+--------------------+
only showing top 5 rows



#### Stage3: Q4: Answer : :
> - The Top 3   precincts are 19,14,1.
    - For Issuer Precinct 19 - Violation Codes are 46,38,37,14 and 21 
    - For Issuer Precinct 14 - Violation Codes are 14,69,31,47 and 42 
    - For Issuer Precinct 1  - Violation Codes are 14,16,20,46 and 38 
> - Only Violation Code which is common across other Issues Preceints (not 19,14 and 1) is 14.

#### 3.5: Question-5:Find out the properties of parking violations across different times of the day:
>-  **Find a way to deal with missing values, if any. <br> (Hint: Check for the null values using 'isNull' under the SQL. Also, to remove the null values, check the 'dropna' command in the API documentation.)**<br><br>
>  **1. The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.**<br><br>
> **2. Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations.
(Hint: Use the CASE-WHEN in SQL view to segregate into bins. To find the most commonly occurring violations, you can use an approach similar to the one mentioned in the hint for question 4.)**<br><br>
> **3. Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).**<br><br>

#### 3.5.1 Check for Null Values

In [35]:
# Verify for NULL values in the data frame using functions.
nyc_parking_tkt_2017_final.select([count(when(col(c).isNull(), c)).alias(c) \
                                   for c in nyc_parking_tkt_2017_final.columns]).show()

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+----------+-----------+
|Summons_Number|Plate_ID|Registration_State|Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|Issue_Year|Issue_Month|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+----------+-----------+
|             0|       0|                 0|         0|             0|                0|           0|                 0|              0|             0|         0|          0|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+----------+-----------+



#### 3.5.2 Fix the format of Violation_Time column and convert the type to 24 hour Timestamp format
> - We can observe that the string format of the time attributes include only a partial component of AM/PM. Therefore we will append M to the end of each time attribute before converting it into a `24 Hour AM/PM timestamp format`.<br><br>
> - Also, following needs to be fixed for Violation_Time Column before converting to appropriate unix time stamp format ;
    - Imputing of all `0000A` and `0000P` in to `1200AM` and `1200PM` respectively.
    - Imputing of all Null values to `1200A`
    - There are values which does not `A` or `P`, so impute them with `A` as default value.
    - Impute column values which are less than 5 characters in to appropriate format
    - Impute column values starting with `00` and `12`
    - Impute special characters such as `+ (plus) , - (hyphen) , . (dot) , (backslash)`.
    - Impute four digit wrong value `015A`,`113 A` and `113 P` to five digits value.

In [36]:
# There are inconsistencies in the Violation_Time Column
# The data to be corrected before converted to appropriate 24 hour Timestamp format

suffix_char = "A"
nyc_parking_tkt_2017_fmt = nyc_parking_tkt_2017_final.withColumn("Violation_Time",\
                            F.when(col("Violation_Time") == '0000A', '1200A')\
                             .when(col("Violation_Time") == '0000P', '1200P')\
                             .when(col("Violation_Time")  ==  'nan', '1200A') \
                             .when(~(nyc_parking_tkt_2017_final["Violation_Time"].contains("A") |\
                                     nyc_parking_tkt_2017_final["Violation_Time"].contains("P")),\
                                    concat(col("Violation_Time"),lit(suffix_char))) \
                             .when(nyc_parking_tkt_2017_final.Violation_Time.substr(0,2) > '12', \
                                                      regexp_replace('Violation_Time','^[0-9]{2}','12'))\
                             .when(col("Violation_Time")  == '015A',  '0150A')\
                             .when(col("Violation_Time")  == '113 A', '1130A')\
                             .when(col("Violation_Time")  == '113 P', '1130P')\
                             .otherwise(col("Violation_Time")) \
                             )
                                                                 
# Remove Special character such +,-,.,`, etc from Violation_Time column.
nyc_parking_tkt_2017_df = nyc_parking_tkt_2017_fmt.withColumn("Violation_Time", \
                                                regexp_replace('Violation_Time','[\+.*$,\.*$,\/.*$,\`.*$]','0'))

# Correct 00XXAM and 00XXPM to appropriate 12XXAM and 12XXPM respectively (24 hour format).
nyc_parking_tkt_2017_df = nyc_parking_tkt_2017_df.withColumn("Violation_Time", \
                                                regexp_replace('Violation_Time','^0{2}','12'))

#Suffix the time having 'A' and 'P' with literal 'M'
suffix_char = "M"
nyc_parking_tkt_2017_df = nyc_parking_tkt_2017_df.withColumn("Violation_Time",\
                                                concat(col("Violation_Time"),lit(suffix_char)))

#Impute 1 row having incorrect timestamp value to appropriate one.
nyc_parking_tkt_2017_df = nyc_parking_tkt_2017_df.withColumn("Violation_Time",\
                        when(col("Violation_Time") == '04080AM', '0408AM').otherwise(col("Violation_Time")))

#### 3.5.3 convert the type to 24 hour Timestamp format

In [37]:
# Convert column Violation_Time to new column Violation_Timestamp with format 24 hour AM/PM Unix Timestamp format .
import pyspark.sql.functions as F
nyc_parking_tkt_2017_df = nyc_parking_tkt_2017_df  \
            .withColumn("Violation_Time_Timestamp", \
                F.from_unixtime(F.unix_timestamp("Violation_Time","hhmma"),'HH:mm:ss a'))

In [38]:
# Validate any invalid translation during conversion into 24 hour format.
nyc_parking_tkt_2017_df.select("Summons_Number","Violation_Time",
                               "Violation_Time_TimeStamp").where("Violation_Time_Timestamp is NULL").show()

+--------------+--------------+------------------------+
|Summons_Number|Violation_Time|Violation_Time_TimeStamp|
+--------------+--------------+------------------------+
+--------------+--------------+------------------------+



In [39]:
nyc_parking_tkt_2017_df.select('Summons_Number','Violation_Time','Violation_Time_Timestamp').show(5)

+--------------+--------------+------------------------+
|Summons_Number|Violation_Time|Violation_Time_Timestamp|
+--------------+--------------+------------------------+
|    1131600721|        0751AM|             07:51:00 AM|
|    1131601841|        1105AM|             11:05:00 AM|
|    1253083198|        0423AM|             04:23:00 AM|
|    1314438890|        0546PM|             17:46:00 PM|
|    1340663107|        1145AM|             11:45:00 AM|
+--------------+--------------+------------------------+
only showing top 5 rows



#### 3.5.4 Create Violation_Hour,Violation_Min and Violation_AMPM for analysis.

In [40]:
# Create new column Violation_Hour,Violation_Min and Violation_AMPM
nyc_parking_tkt_2017_df = nyc_parking_tkt_2017_df\
           .withColumn("Violation_Hour",nyc_parking_tkt_2017_df.Violation_Time_Timestamp.substr(0,2).cast("int")) \
           .withColumn("Violation_Min",nyc_parking_tkt_2017_df.Violation_Time_Timestamp.substr(4,2).cast("int"))  \
           .withColumn("Violation_AMPM",nyc_parking_tkt_2017_df.Violation_Time_Timestamp.substr(10,2)) \
        
# Check the results
nyc_parking_tkt_2017_df.select("Violation_Time","Violation_Time_Timestamp","Violation_Hour",
                                          "Violation_Min","Violation_AMPM").show(20)

+--------------+------------------------+--------------+-------------+--------------+
|Violation_Time|Violation_Time_Timestamp|Violation_Hour|Violation_Min|Violation_AMPM|
+--------------+------------------------+--------------+-------------+--------------+
|        0751AM|             07:51:00 AM|             7|           51|            AM|
|        1105AM|             11:05:00 AM|            11|            5|            AM|
|        0423AM|             04:23:00 AM|             4|           23|            AM|
|        0546PM|             17:46:00 PM|            17|           46|            PM|
|        1145AM|             11:45:00 AM|            11|           45|            AM|
|        1210AM|             00:10:00 AM|             0|           10|            AM|
|        0111PM|             13:11:00 PM|            13|           11|            PM|
|        1021AM|             10:21:00 AM|            10|           21|            AM|
|        0810AM|             08:10:00 AM|             

In [41]:
nyc_parking_tkt_2017_df.select("Violation_Hour").distinct().sort("Violation_Hour",ascending=False).show(25)

+--------------+
|Violation_Hour|
+--------------+
|            23|
|            22|
|            21|
|            20|
|            19|
|            18|
|            17|
|            16|
|            15|
|            14|
|            13|
|            12|
|            11|
|            10|
|             9|
|             8|
|             7|
|             6|
|             5|
|             4|
|             3|
|             2|
|             1|
|             0|
+--------------+



#### 3.5.5  Divide 24 hours into six equal discrete bins of time.

In [42]:
# Recreate the Temp View after Fixing Violation_Time  Format.
nyc_parking_tkt_2017_df.createOrReplaceTempView("tbl_nyctkt_2017")

In [43]:
# Divide 24 hours into 6 equal discrete bins of time. The intervals you choose are at your discretion.

# SQL Query - sql_18
sql_18 = "SELECT Violation_Hour,Violation_Code,\
            CASE \
                WHEN Violation_Hour BETWEEN  0 AND  3 THEN 'bin_0_to_3'   \
                WHEN Violation_Hour BETWEEN  4 AND  7 THEN 'bin_4_to_7'   \
                WHEN Violation_Hour BETWEEN  8 AND 11 THEN 'bin_8_to_11'  \
                WHEN Violation_Hour BETWEEN 12 AND 15 THEN 'bin_12_to_15' \
                WHEN Violation_Hour BETWEEN 16 AND 19 THEN 'bin_16_to_19' \
                WHEN Violation_Hour BETWEEN 20 AND 23 THEN 'bin_20_to_23' \
                END AS Violation_Hour_Bin \
          FROM tbl_nyctkt_2017"


#Execute the Query and list out .
sql_18_result = spark.sql(sql_18)
sql_18_result.show(5)

+--------------+--------------+------------------+
|Violation_Hour|Violation_Code|Violation_Hour_Bin|
+--------------+--------------+------------------+
|             7|            17|        bin_4_to_7|
|            11|            74|       bin_8_to_11|
|             4|            21|        bin_4_to_7|
|            17|            14|      bin_16_to_19|
|            11|            67|       bin_8_to_11|
+--------------+--------------+------------------+
only showing top 5 rows



#### 3.5.6  Create Temporary View Table for analysis.

In [44]:
# Create Temporary View Table based on the resultset.
sql_18_result.createOrReplaceTempView("tbl_violation_hour_bin")

#### 3.5.7  Query to find the three most commonly occurring violations (based on above Violation_Hour_Bins)

In [45]:
# SQL Query - sql_19
sql_19 = "SELECT Violation_Hour_Bin, \
                 Violation_Code, \
                 Frequency_of_Tickets, \
                 Rnk \
                 FROM(SELECT \
                        A.Violation_Hour_Bin, \
                        A.Violation_Code, \
                        A.Frequency_of_Tickets, \
                        DENSE_RANK() OVER(PARTITION BY A.Violation_Hour_Bin ORDER BY A.Frequency_of_Tickets DESC) AS Rnk \
                    FROM (SELECT \
                              Violation_Hour_Bin, \
                              Violation_Code, \
                              COUNT(*) AS Frequency_of_Tickets \
                          FROM tbl_violation_hour_bin \
                          GROUP BY Violation_Hour_Bin, Violation_Code ) AS A \
                     ) \
                 WHERE Rnk <=3"  
            
#Execute the Query and list out .
sql_19_result = spark.sql(sql_19)
sql_19_result.show()

+------------------+--------------+--------------------+---+
|Violation_Hour_Bin|Violation_Code|Frequency_of_Tickets|Rnk|
+------------------+--------------+--------------------+---+
|        bin_0_to_3|            21|               36958|  1|
|        bin_0_to_3|            40|               25867|  2|
|        bin_0_to_3|            78|               15528|  3|
|      bin_16_to_19|            38|              102855|  1|
|      bin_16_to_19|            14|               75902|  2|
|      bin_16_to_19|            37|               70345|  3|
|        bin_4_to_7|            14|               74114|  1|
|        bin_4_to_7|            40|               60652|  2|
|        bin_4_to_7|            21|               57897|  3|
|      bin_20_to_23|             7|               26293|  1|
|      bin_20_to_23|            40|               22337|  2|
|      bin_20_to_23|            14|               21045|  3|
|       bin_8_to_11|            21|              598070|  1|
|       bin_8_to_11|    

#### 3.5.8  Query to find out Three most commonly occurring violation codes in the year 2017.

In [46]:
# SQL Query - sql_20
sql_20 = "SELECT Violation_Code, COUNT(*) AS Number_of_Tickets \
          FROM tbl_violation_hour_bin \
          GROUP BY Violation_Code \
          ORDER BY Number_of_Tickets DESC"

#Execute the Query and list out .
sql_20_result = spark.sql(sql_20)
sql_20_result.show(3)

+--------------+-----------------+
|Violation_Code|Number_of_Tickets|
+--------------+-----------------+
|            21|           768087|
|            36|           662765|
|            38|           542079|
+--------------+-----------------+
only showing top 3 rows



***Observations***<br>

- The Top three Violation Codes based on Number of Tickets for the Year 2017 are `21`, `36` and `38`.

#### 3.5.9  Query Find out most common time of the day these three (21,36 and 38).(based on bins from above)

In [47]:
# SQL Query - sql_21
sql_21 = "SELECT Violation_Code, Violation_Hour_Bin, COUNT(*) AS Number_of_Tickets \
          FROM tbl_violation_hour_bin \
          WHERE Violation_Code IN (21,36,38) \
          GROUP BY Violation_Code,Violation_Hour_Bin \
          ORDER BY Violation_Code,Violation_Hour_Bin,Number_of_Tickets DESC"

#Execute the Query and list out .
sql_21_result = spark.sql(sql_21)
sql_21_result.show()

+--------------+------------------+-----------------+
|Violation_Code|Violation_Hour_Bin|Number_of_Tickets|
+--------------+------------------+-----------------+
|            21|        bin_0_to_3|            36958|
|            21|      bin_12_to_15|            74719|
|            21|      bin_16_to_19|              259|
|            21|      bin_20_to_23|              184|
|            21|        bin_4_to_7|            57897|
|            21|       bin_8_to_11|           598070|
|            36|      bin_12_to_15|           286284|
|            36|      bin_16_to_19|            13534|
|            36|        bin_4_to_7|            14782|
|            36|       bin_8_to_11|           348165|
|            38|        bin_0_to_3|              312|
|            38|      bin_12_to_15|           240722|
|            38|      bin_16_to_19|           102855|
|            38|      bin_20_to_23|            20347|
|            38|        bin_4_to_7|             1273|
|            38|       bin_8

#### Stage3: Q5: Answer : :
> -  **There are no missing or NULL Values in the data.**<br>
>  **1. The Violation Time column is cleaned and formatted correctly , so that it is converted to apprproiate 24 hour timestamp format. New columns Violation_Hours, Violation_Min and Violation_AMPM created for analysis.**<br>
>  **2.  The data is divided in to 6 different bins based on Violation_Hour (3 hour window) using SQL Query using CASE..WHEN.**<br>
>  **3. Finally, 3 most commonly occuring Violation Codes identified, based each of the bins obtained from step 2.**

> -  The resultset for the three most commonly occuring violation codes for based on each of bins and their ranking as below;
                    +------------------+--------------+--------------------+---+
                    |Violation_Hour_Bin|Violation_Code|Frequency_of_Tickets|Rnk|
                    +------------------+--------------+--------------------+---+
                    |        bin_0_to_3|            21|               36958|  1|
                    |        bin_0_to_3|            40|               25867|  2|
                    |        bin_0_to_3|            78|               15528|  3|
                    |      bin_16_to_19|            38|              102855|  1|
                    |      bin_16_to_19|            14|               75902|  2|
                    |      bin_16_to_19|            37|               70345|  3|
                    |        bin_4_to_7|            14|               74114|  1|
                    |        bin_4_to_7|            40|               60652|  2|
                    |        bin_4_to_7|            21|               57897|  3|
                    |      bin_20_to_23|             7|               26293|  1|
                    |      bin_20_to_23|            40|               22337|  2|
                    |      bin_20_to_23|            14|               21045|  3|
                    |       bin_8_to_11|            21|              598070|  1|
                    |       bin_8_to_11|            36|              348165|  2|
                    |       bin_8_to_11|            38|              176570|  3|
                    |      bin_12_to_15|            36|              286284|  1|
                    |      bin_12_to_15|            38|              240722|  2|
                    |      bin_12_to_15|            37|              167026|  3|
                    +------------------+--------------+--------------------+---+                 
> - The three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).
                            +--------------+------------------+-----------------+
                            |Violation_Code|Violation_Hour_Bin|Number_of_Tickets|
                            +--------------+------------------+-----------------+
                            |            21|        bin_0_to_3|            36958|
                            |            21|      bin_12_to_15|            74719|
                            |            21|      bin_16_to_19|              259|
                            |            21|      bin_20_to_23|              184|
                            |            21|        bin_4_to_7|            57897|
                            |            21|       bin_8_to_11|           598070|
                            |            36|      bin_12_to_15|           286284|
                            |            36|      bin_16_to_19|            13534|
                            |            36|        bin_4_to_7|            14782|
                            |            36|       bin_8_to_11|           348165|
                            |            38|        bin_0_to_3|              312|
                            |            38|      bin_12_to_15|           240722|
                            |            38|      bin_16_to_19|           102855|
                            |            38|      bin_20_to_23|            20347|
                            |            38|        bin_4_to_7|             1273|
                            |            38|       bin_8_to_11|           176570|
                            +--------------+------------------+-----------------+

#### 3.6: Question-6:Let’s try and find some seasonality in this data:<br>
>- ** First, divide the year into a certain number of seasons, and find the frequencies of tickets for each season. (Hint: Use Issue Date to segregate into seasons.)**<br>

>- ** Then, find the three most common violations for each of these seasons.
(Hint: You can use an approach similar to the one mentioned in the hint for question 4.)** </br>

***Observations***
> #### Checking for seasonality in dataset :#### 
> - ***Divide the Issue_Year in to 4 Seasons - Winter, Spring, Summer and Fall.*** 
> - ***Find the Frequency of Tickets in each of the Season.***
> - ***Then, find the three most common violations for each of these seasons.***
> - **Note:** We have already split the `Issue_Date` in to `Issue_Year` and `Issue_Month` in the `Stage-1 [Section:1.9]`

#### 3.6.1 Query to Divide the Issue_Month in to 4 Seasons.

In [48]:
# SQL Query - sql_22
sql_22 = "SELECT Summons_Number, Violation_Code,\
          CASE \
             WHEN Issue_Month IN (1,2,12)      THEN 'Winter' \
             WHEN Issue_Month BETWEEN 3 AND 5  THEN 'Spring' \
             WHEN Issue_Month BETWEEN 6 AND 8  THEN 'Summer' \
             WHEN Issue_Month BETWEEN 9 AND 11 THEN 'Fall'   \
          END AS Season \
          FROM tbl_nyctkt_2017"

#Execute the Query and list out .
sql_22_result = spark.sql(sql_22)
sql_22_result.show(5)

+--------------+--------------+------+
|Summons_Number|Violation_Code|Season|
+--------------+--------------+------+
|    8539607141|            46|Spring|
|    8491667738|            19|Spring|
|    8514570973|            21|Summer|
|    8348055757|            21|Spring|
|    8471227990|            50|Spring|
+--------------+--------------+------+
only showing top 5 rows



#### 3.6.2 Create Temporary View Table for analysis.

In [49]:
# Create the Temp View .
sql_22_result.createOrReplaceTempView("tbl_seasons_nyctkt_2017")

#### 3.6.3 Query to Find the Frequency of Tickets in each of the Seasons in Year 2017 from the Temporary View Table.

In [50]:
# SQL Query - sql_23
sql_23 = "SELECT Season,COUNT(*) AS Frequency_of_Tickets \
          FROM tbl_seasons_nyctkt_2017 \
          GROUP BY Season \
          ORDER BY Frequency_of_Tickets DESC "

#Execute the Query and list out .
sql_23_result = spark.sql(sql_23)
sql_23_result.show()

+------+--------------------+
|Season|Frequency_of_Tickets|
+------+--------------------+
|Spring|             2873383|
|Winter|             1704690|
|Summer|              852866|
|  Fall|                 979|
+------+--------------------+



#### 3.6.4 Query to find the three most common violations for each of the seasons as above.

In [51]:
# SQL Query - sql_24
sql_24 = "SELECT Season, \
                 Violation_Code, \
                 Frequency_of_Tickets, \
                 Rank \
          FROM (SELECT DENSE_RANK() OVER(PARTITION BY A.Season ORDER BY A.Frequency_of_Tickets DESC) AS Rank, \
                       A.Season, \
                       A.Violation_Code, \
                       A.Frequency_of_Tickets \
                FROM (SELECT \
                         Season, \
                         Violation_Code, \
                         COUNT(*) AS Frequency_of_Tickets \
                      FROM tbl_seasons_nyctkt_2017 \
                      GROUP BY Season, Violation_Code \
                     ) AS A \
                ) \
           WHERE Rank <= 3"
         
#Execute the Query and list out .
sql_24_result = spark.sql(sql_24)
sql_24_result.show()

+------+--------------+--------------------+----+
|Season|Violation_Code|Frequency_of_Tickets|Rank|
+------+--------------+--------------------+----+
|Spring|            21|              402424|   1|
|Spring|            36|              344834|   2|
|Spring|            38|              271167|   3|
|Summer|            21|              127352|   1|
|Summer|            36|               96663|   2|
|Summer|            38|               83518|   3|
|  Fall|            46|                 231|   1|
|  Fall|            21|                 128|   2|
|  Fall|            40|                 116|   3|
|Winter|            21|              238183|   1|
|Winter|            36|              221268|   2|
|Winter|            38|              187386|   3|
+------+--------------+--------------------+----+



#### Stage3: Q6: Answer : :
> -  **Divided the Issue Month in 4 Season bins as `Winter`,`Spring`,`Summer` and `Fall`.** 
> - **The Frequencies of Tickets for each of the seasons  are as below ;**
                    +------+--------------------+
                    |Season|Frequency_of_Tickets|
                    +------+--------------------+
                    |Spring|             2873383|
                    |Winter|             1704690|
                    |Summer|              852866|
                    |  Fall|                 979|
                    +------+--------------------+
>- **The three most common violations for each of the season are as below;**
                    +------+--------------+--------------------+----+
                    |Season|Violation_Code|Frequency_of_Tickets|Rank|
                    +------+--------------+--------------------+----+
                    |Spring|            21|              402424|   1|
                    |Spring|            36|              344834|   2|
                    |Spring|            38|              271167|   3|
                    |Summer|            21|              127352|   1|
                    |Summer|            36|               96663|   2|
                    |Summer|            38|               83518|   3|
                    |  Fall|            46|                 231|   1|
                    |  Fall|            21|                 128|   2|
                    |  Fall|            40|                 116|   3|
                    |Winter|            21|              238183|   1|
                    |Winter|            36|              221268|   2|
                    |Winter|            38|              187386|   3|
                    +------+--------------+--------------------+----+

#### 3.6: Question-7:The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department. Let’s take an example of estimating this for the three most commonly occurring codes:<br>
>**1. Find the total occurrences of the three most common violation codes.**<br>
>**2. Then, visit the website:http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page. It lists the fines associated with different violation codes. They’re divided into two categories:**
>   - **one for the highest-density locations in the city and the other for the rest of the city.**
>   - **For the sake of simplicity, take the average of the two.**

>**3. Using this information, find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.**<br>
>**4. What can you intuitively infer from these findings?**<br>


#### 3.7.1 Query to Find the total occurrences of the three most common violation codes.

In [52]:
# SQL Query - sql_25
sql_25 = "SELECT Violation_Code,COUNT(*) AS Frequency_of_Tickets \
          FROM tbl_nyctkt_2017 \
          GROUP BY Violation_Code \
          ORDER BY Frequency_of_Tickets DESC LIMIT 3"

#Execute the Query and list out .
Top3_violation_code_df = spark.sql(sql_25)
Top3_violation_code_df.show()

+--------------+--------------------+
|Violation_Code|Frequency_of_Tickets|
+--------------+--------------------+
|            21|              768087|
|            36|              662765|
|            38|              542079|
+--------------+--------------------+



***Observations***
>- **The most common violation codes based on number of tickets received on Year 2017 are - `21`,`36` and `38` **<br>
>- **The values below shows the parking violation codes in New York City and their fines.**<br>
>    -  **Code `21`:  For Manhattan 96th St. & below - `$65`  ; All Other Areas:  `$45` ; Average - `$55.00`**
>    -  **Code `36`:  For Manhattan 96th St. & below - `$50`  ; All Other Areas:  `$50` ; Average - `$50.00`**
>    -  **Code `38`:  For Manhattan 96th St. & below - `$65`  ; All Other Areas:  `$35` ; Average - `$50.00`**


#### 3.7.2 Create a Data Frame with Violation Codes and Associated Fines (for High Density Locations & Others).

In [53]:
#  Create data frame for Fine Amounts and Average Amount

# List to holds fine code and fine amounts.
lst_code = [21,36,38]
lst_m96_st = [65.0,50.0,65.0]
lst_others = [45.0,50.0,35.0]

# create a schema.
from pyspark.sql.types import *
myschema= StructType([ StructField("Violation_Code", IntegerType(), True),
                       StructField("M96st_Fine_Per_Tkt", DoubleType(), True),
                       StructField("Others_Fine_Per_Tkt", DoubleType(), True),
                     ])

# Create a Top3 fine amount data frame.
Top3_Fine_Amt_2017 = spark.createDataFrame(zip(lst_code,lst_m96_st,lst_others),schema = myschema)

# Calculate the Average Amount
Top3_Fine_Amt_2017 = Top3_Fine_Amt_2017.withColumn("Avg_Fine_Per_Tkt", \
                    (Top3_Fine_Amt_2017['M96st_Fine_Per_Tkt'] +Top3_Fine_Amt_2017['Others_Fine_Per_Tkt'])/2.0)

Top3_Fine_Amt_2017.show()

+--------------+------------------+-------------------+----------------+
|Violation_Code|M96st_Fine_Per_Tkt|Others_Fine_Per_Tkt|Avg_Fine_Per_Tkt|
+--------------+------------------+-------------------+----------------+
|            21|              65.0|               45.0|            55.0|
|            36|              50.0|               50.0|            50.0|
|            38|              65.0|               35.0|            50.0|
+--------------+------------------+-------------------+----------------+



In [54]:
Top3_violation_code_df.registerTempTable("tbl_violation")
Top3_Fine_Amt_2017.registerTempTable("tbl_fine")

#### 3.7.2 Join the two Data Frames - 3 most common Violation Codes & Top 3 fine details.

In [55]:
join_sql = "SELECT tbl_violation.Violation_Code,\
                   tbl_violation.Frequency_of_Tickets,\
                   tbl_fine.Avg_Fine_Per_Tkt \
               FROM tbl_violation JOIN tbl_fine \
                    ON tbl_violation.Violation_Code == tbl_fine.Violation_Code"

#Execute the Query and list out .
Top3_total_Amt_2017 = spark.sql(join_sql)
Top3_total_Amt_2017.show()

+--------------+--------------------+----------------+
|Violation_Code|Frequency_of_Tickets|Avg_Fine_Per_Tkt|
+--------------+--------------------+----------------+
|            21|              768087|            55.0|
|            36|              662765|            50.0|
|            38|              542079|            50.0|
+--------------+--------------------+----------------+



In [56]:
# Calculate the Total Amount create the new column.
Top3_total_Amt_2017 = Top3_total_Amt_2017.withColumn("Total_Fine_Amt_2017", \
                    (Top3_total_Amt_2017['Frequency_of_Tickets'] * Top3_total_Amt_2017['Avg_Fine_Per_Tkt'])\
                                                    .cast(DecimalType(18, 2)))
                                                     
Top3_total_Amt_2017.show()

+--------------+--------------------+----------------+-------------------+
|Violation_Code|Frequency_of_Tickets|Avg_Fine_Per_Tkt|Total_Fine_Amt_2017|
+--------------+--------------------+----------------+-------------------+
|            21|              768087|            55.0|        42244785.00|
|            36|              662765|            50.0|        33138250.00|
|            38|              542079|            50.0|        27103950.00|
+--------------+--------------------+----------------+-------------------+



#### Stage3: Q7: Answer : :
> -  **The total occurrences of the three most common violation codes are ;**
                    +--------------+--------------------+
                    |Violation_Code|Frequency_of_Tickets|
                    +--------------+--------------------+
                    |            21|              768087|
                    |            36|              662765|
                    |            38|              542079|
                    +--------------+--------------------+
 > - **The Top three violation code associated fines and their averages are as below ;**
                     +--------------+------------------+-------------------+----------------+
                    |Violation_Code|M96st_Fine_Per_Tkt|Others_Fine_Per_Tkt|Avg_Fine_Per_Tkt|
                    +--------------+------------------+-------------------+----------------+
                    |            21|              65.0|               45.0|            55.0|
                    |            36|              50.0|               50.0|            50.0|
                    |            38|              65.0|               35.0|            50.0|
                    +--------------+------------------+-------------------+----------------+
 > - **The total amount collected for the three violation codes with the maximum tickets are;**
                     +--------------+--------------------+----------------+-------------------+
                    |Violation_Code|Frequency_of_Tickets|Avg_Fine_Per_Tkt|Total_Fine_Amt_2017|
                    +--------------+--------------------+----------------+-------------------+
                    |            21|              768087|            55.0|        42244785.00|
                    |            36|              662765|            50.0|        33138250.00|
                    |            38|              542079|            50.0|        27103950.00|
                    +--------------+--------------------+----------------+-------------------+

#### 3.7: Question-7: What can you intuitively infer from these findings?
> - The inference based on all above questions and finding are as below ;

####  Answer : :
###   The inference based on all above questions and finding are as below ;

##### A. Examine the data
>  ** 1. The number of parking tickets issued during year 2017 is approximately `5,431,918 Million Tickets`. (i.e, `5.4 Million Tickets`).**

>  ** 2. The 	number of unique states of the registered plate of cars receiving parking tickets in the year are 65. However, there are 16055 records with state code 99.This implies that there is several parking tickets issued to vehicle less with an unknown registration state. For the sake of case study, we considered these unknown state code as 'NY'. So, the maximum parking tickets received by vehicles having registered plates are `64` and the state is 'NY'. **<br><br>
     
###### B. Aggregation tasks<br>

**1. The frequency of top 5 Violation Codes high during the year 2017 are for ;** 
- ** `21` - No parking where parking is not allowed by sign, street marking or traffic control device. ** 
- ** `36` - Exceeding the posted speed limit in or near a designated school zone. **<br>
- ** `38` - Failing to show a receipt or tag in the windshield. Drivers get a 5-minute grace period past the expired time on parking meter receipts.**
- ** `14` - Standing or parking where standing is not allowed by sign, street marking or; traffic control device.**<br>
- ** `20` - No parking where parking is not allowed by sign, street marking or traffic control device.**<br><br>
- **Therefore, the highest frequency of tickets through the years is issued for No-parking violations, Exceeding Speed limit near school zones and Failing to show tag in the windsheild.** <br>
     
**2. The `top 5 Vehicle Body Types issued with parking tickets in 2017` are ;**
- ** `SUBN` - Vehicles that can be registered with the suburban body type include station wagons, sport utility vehicles, hearses and ambulances.**
- ** `4DSD` - 4 door Sedans**
- ** `VAN`  - Type of vehicle used for transporting goods or people.**
- ** `DELV` - Delivery Truck**
- ** `SDN`  - Civilian Sedan**<br><br>

- ** The top 5 Vehicle Make having issued with parking tickets in 2017 are ;**<br>
     - ** Ford, Toyota, Honda , Nissan and Cheverlet.**<br>
     
** 3.1. From the analysis of violation of Precinct against the number of parking tickets ;**<br>
   - **it is clear the highest number of tickets were issued with Violation Precinct 0. Since 0 is unknown precinct it may be issued with faulty tickets. The Violation Precinct codes are `19,14 and 1`.**
 
      
** 3.2. From the analysis of Issues Precinct against the number of parking tickets ;**<br>  
- ** Reveals the highest number of tickets are issued for code `0`. Again, there is no code as 0 , the validity of the tickets raised are questionable. Other top issue Precinct codes are `19,14 and 1` .**

   
**4. While  observing the Top-5 violation codes for which tickets were issued in each of the above 3 Issuing Precincts (19,14 and 1) and comparing the results with all other Issuing Precincts for the year 2017, the`violation codes 21 and 36 have exceptionally high count of tickets`.**


**5.1. There are no NULL values in the data.**

**5.2. The Violation Time has been transformed into a suitable timestamp format and Divide 24 hours into 6 equal discrete bins of time.**

**5.3. While observing the Frequency of Top-3 Violation Codes in each Violation Time Bin it is clear
that the majority of the Tickets are issued between `0800-1100 Hrs and 1200-1500 Hrs`. It is also important to note,the exceptionally high number of tickets for `Violation Code 21` This is expected as Code 21 stands for No- Parking Zone Tickets. Majority of the public might park inappropriately during the morning rush, issued between `0800-1100 Hrs`.**

**5.4. The Top-3 Violation Codes for 2016 are;<br>**
  - **21: No Parking Violation **
  - **36: Exceeding speed limit near School Zone**
  - **38: Failing to show a receipt or tag in the windshield**<br>
  - **While looking at the distribution of Violation Code 21, there is noticeably a `significant peak between 0800-1100 Hrs and dips between 1600-2300 Hrs`. For `Violation Code 36` there are almost `equal peaks between 0800-1100 Hrs and 1200-1500 Hrs` this may be as a result of people rushing to drop and pickup children to and from school.**
  
**6. After dividing the Issue Month into number of seasons, for each season and for the year 2017, Spring and Winter account for the highest number of parking tickets.**<br>
  
**7.1. The Top three most Violation Codes are ;<br>**
 - ** 21: No Parking Violation,**
 - ** 36: Exceeding speed limit near School Zone,**
 - ** 38: Failing to show a receipt or tag in the windshield**

**7.2. In the year  2017,`Violation Code 21` brought the `highest total fine amount $42,244,785.00 Millions(42.2 Millions`.**

In [57]:
# Stop the Spark Session to come out cleanly.
#spark.stop()